In [1]:
import pyxem as pxm
import pymatgen as pmg
import numpy as np
from pyxem.generators.indexation_generator import IndexationGenerator
from pyxem.utils.plot import generate_marker_inputs_from_peaks
from pyxem.utils.sim_utils import peaks_from_best_template
import hyperspy.api as hs
%matplotlib tk

In [2]:
if True:
    sample = pxm.load('/home/pc494/Documents/data/cBN/160401_MultipleZoneAxes/160401_03.blo')
    #sample = sample.inav[50:-60,15:130]
    
half_side_length = (512/2)

In [3]:
## get an undeformed section
dp = pxm.ElectronDiffraction(sample.inav[13:18,102:107])

#CORRECTING FOR CAMERA
dp.apply_affine_transformation(np.array([[0.99,0,0],
                                         [0,0.69,0],
                                         [0,0,1]]))
#CORRECTING FOR BEING OFF CENTER
dp.get_direct_beam_position(method='blur',sigma=10,progress_bar=False) #checking centers, also BUG!
dp.apply_affine_transformation(D=np.array([[1,0,0],
                                           [0,1,3],
                                           [0,0,1]]))

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [4]:
#dp.plot()

In [5]:
pixel_distance = 79.6# np.sqrt(50**2 + 61.7**2)

In [6]:
theoretical_distance = 1/np.sqrt((3.615**2)/(8))

In [9]:
dp.set_calibration(theoretical_distance/pixel_distance)

In [10]:
a = 3.615
B = pmg.Element("B")
N = pmg.Element("N")
lattice = pmg.Lattice.cubic(a)
structure = pmg.Structure.from_spacegroup("F-43m",lattice, [B,N], [[0, 0, 0],[0.25,0.25,0.25]])

In [11]:
def build_structure_lib(structure,rot_list):
    struc_lib = dict()
    struc_lib["A"] = (structure,rot_list)
    return struc_lib    

def build_linear_grid_in_euler(alpha_min,alpha_max,beta_min,beta_max,gamma_min,gamma_max,resolution):
    a = np.arange(alpha_min,alpha_max,step=resolution)
    b = np.arange(beta_min,beta_max,step=resolution)
    c = np.arange(gamma_min,gamma_max,step=resolution)
    from itertools import product
    return list(product(a,b,c))

In [12]:
edc = pxm.DiffractionGenerator(300, 5e-2)
diff_gen = pxm.DiffractionLibraryGenerator(edc)
rot_list = build_linear_grid_in_euler(0,90,0,90,0,90,5) 
struc_lib = build_structure_lib(structure,rot_list)
library = diff_gen.get_diffraction_library(struc_lib,
                                           calibration=theoretical_distance/pixel_distance,
                                           reciprocal_radius=1.5,
                                           half_shape=(half_side_length,half_side_length),
                                           representation='euler',
                                           with_direct_beam=False)

In [13]:
indexer = IndexationGenerator(dp,library)
match_results = indexer.correlate()

A Jupyter Widget

In [14]:
peaks = match_results.map(peaks_from_best_template,phase=["A"],library=library,inplace=False)
mmx,mmy = generate_marker_inputs_from_peaks(peaks)
dp.plot(cmap='viridis')
for mx,my in zip(mmx,mmy):
    m = hs.markers.point(x=mx,y=my,color='yellow',marker='x') #see visual test
    dp.add_marker(m,plot_marker=True,permanent=True)

A Jupyter Widget

In [18]:
match_results.inav[0,0].data

array([[  0.        ,  26.        ,  65.        ,  84.        ,
          1.15082964],
       [  0.        ,  25.        ,  87.        ,  84.        ,
          1.13752023],
       [  0.        ,  26.        ,  65.        ,  73.        ,   1.1274638 ],
       [  0.        ,  25.        ,  87.        ,  73.        ,
          1.12699677],
       [  0.        ,   3.        ,  87.        ,  84.        ,
          1.12623112]])

In [16]:
p = library["A"][(26,180,40)]['Sim']
p.indices

KeyError: (26, 180, 40)

In [ ]:
from transforms3d.euler import euler2mat

t = euler2mat(26,180,40, 'rzxz')
np.matmul(t,[0,0,1])

In [20]:
len(rot_list)

729000